<a href="https://colab.research.google.com/github/singhpratham19/fsd/blob/main/olist_buisness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

# Create a database file inside Colab
conn = sqlite3.connect("olist.db")

print("Database created successfully!")

Database created successfully!


In [3]:
import pandas as pd

orders = pd.read_csv("olist_orders_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")
reviews = pd.read_csv("olist_order_reviews_dataset.csv")

In [4]:
orders.head()
customers.head()
orders.shape

(99441, 8)

In [5]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [6]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [7]:
import pandas as pd

# Load orders with datetime parsing
orders = pd.read_csv(
    "olist_orders_dataset.csv",
    parse_dates=[
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date"
    ]
)

customers = pd.read_csv("olist_customers_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")
reviews = pd.read_csv("olist_order_reviews_dataset.csv")

In [8]:
df = orders.merge(customers, on="customer_id", how="left")
df.shape

(99441, 12)

In [9]:
df = df.merge(order_items, on="order_id", how="left")
df.shape

(113425, 18)

In [10]:
df = df.merge(products[['product_id', 'product_category_name']], on="product_id", how="left")
df.shape

(113425, 19)

In [11]:
df = df.merge(payments, on="order_id", how="left")
df.shape

(118434, 23)

In [12]:
df = df.merge(reviews[['order_id', 'review_score']], on="order_id", how="left")
df.shape

(119143, 24)

In [13]:
df.shape

(119143, 24)

In [14]:
df = df[df["order_delivered_customer_date"].notna()]
df.shape

(115722, 24)

In [15]:
df["delivery_days"] = (
    df["order_delivered_customer_date"] - df["order_purchase_timestamp"]
).dt.days

In [16]:
df["estimated_delivery_days"] = (
    df["order_estimated_delivery_date"] - df["order_purchase_timestamp"]
).dt.days

In [17]:
df["is_late_delivery"] = df["delivery_days"] > df["estimated_delivery_days"]

In [18]:
df["total_price"] = df["price"] + df["freight_value"]

In [19]:
df["order_month"] = df["order_purchase_timestamp"].dt.to_period("M").astype(str)

In [20]:
df = df[df["delivery_days"] >= 0]

In [21]:
df = df[df["delivery_days"] <= 60]

In [22]:
df = df[df["total_price"].notna()]

In [23]:
df.shape

(115394, 29)

In [24]:
df.to_csv("cleaned_olist_data.csv", index=False)

In [25]:
from google.colab import files
files.download("cleaned_olist_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
import sqlite3

conn = sqlite3.connect("olist.db")

df.to_sql("olist_clean", conn, if_exists="replace", index=False)

print("SQLite table 'olist_clean' created!")

SQLite table 'olist_clean' created!


In [27]:
pd.read_sql("SELECT * FROM olist_clean LIMIT 5;", conn)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,payment_sequential,payment_type,payment_installments,payment_value,review_score,delivery_days,estimated_delivery_days,is_late_delivery,total_price,order_month
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,1.0,credit_card,1.0,18.12,4.0,8,15,0,38.71,2017-10
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,3.0,voucher,1.0,2.00,4.0,8,15,0,38.71,2017-10
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,2.0,voucher,1.0,18.59,4.0,8,15,0,38.71,2017-10
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,...,1.0,boleto,1.0,141.46,4.0,13,19,0,141.46,2018-07
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,1.0,credit_card,3.0,179.12,5.0,9,26,0,179.12,2018-08


In [28]:
pd.read_sql("""
SELECT customer_state, COUNT(*) AS total_orders
FROM olist_clean
GROUP BY customer_state
ORDER BY total_orders DESC;
""", conn)

,customer_state,total_orders
0,SP,48747
1,RJ,14865
2,MG,13487
3,RS,6418
4,PR,5888
5,SC,4232
6,BA,3916
7,DF,2445
8,GO,2389
9,ES,2309


In [29]:
pd.read_sql("""
SELECT product_category_name, SUM(total_price) AS revenue
FROM olist_clean
GROUP BY product_category_name
ORDER BY revenue DESC;
""", conn)

,product_category_name,revenue
0,beleza_saude,1456002.03
1,relogios_presentes,1313545.65
2,cama_mesa_banho,1305468.28
3,esporte_lazer,1162907.71
4,informatica_acessorios,1074254.44
...,...,...
69,pc_gamer,1575.17
70,casa_conforto_2,1194.44
71,cds_dvds_musicais,954.99
72,fashion_roupa_infanto_juvenil,598.67


In [30]:
pd.read_sql("""
SELECT order_month, COUNT(DISTINCT order_id) AS total_orders
FROM olist_clean
GROUP BY order_month
ORDER BY order_month;
""", conn)

,order_month,total_orders
0,2016-09,1
1,2016-10,267
2,2016-12,1
3,2017-01,744
4,2017-02,1647
5,2017-03,2526
6,2017-04,2295
7,2017-05,3538
8,2017-06,3125
9,2017-07,3864


In [31]:
pd.read_sql("""
SELECT AVG(delivery_days) AS avg_delivery_days
FROM olist_clean;
""", conn)

,avg_delivery_days
0,11.806342


In [32]:
pd.read_sql("""
SELECT AVG(delivery_days) AS avg_delivery_days
FROM olist_clean;
""", conn)

,avg_delivery_days
0,11.806342


In [33]:
pd.read_sql("""
SELECT customer_state,
       AVG(CASE WHEN is_late_delivery = 1 THEN 1 ELSE 0 END)*100 AS late_percentage
FROM olist_clean
GROUP BY customer_state
ORDER BY late_percentage DESC;
""", conn)

,customer_state,late_percentage
0,AL,23.076923
1,MA,19.070905
2,SE,15.143603
3,PI,14.545455
4,CE,13.418746
5,BA,12.104188
6,RJ,11.772620
7,ES,11.346903
8,TO,11.077844
9,PA,11.059908


In [34]:
pd.read_sql("""
SELECT customer_state,
       AVG(CASE WHEN is_late_delivery = 1 THEN 1 ELSE 0 END)*100 AS late_percentage
FROM olist_clean
GROUP BY customer_state
ORDER BY late_percentage DESC;
""", conn)

,customer_state,late_percentage
0,AL,23.076923
1,MA,19.070905
2,SE,15.143603
3,PI,14.545455
4,CE,13.418746
5,BA,12.104188
6,RJ,11.772620
7,ES,11.346903
8,TO,11.077844
9,PA,11.059908


In [35]:
pd.read_sql("""
SELECT review_score, COUNT(*) AS total_reviews
FROM olist_clean
GROUP BY review_score
ORDER BY review_score;
""", conn)

,review_score,total_reviews
0,NaN,844
1,1.0,12942
2,2.0,3882
3,3.0,9623
4,4.0,22115
5,5.0,65988


In [36]:
state_orders = pd.read_sql("""SELECT customer_state, COUNT(*) AS total_orders
FROM olist_clean GROUP BY customer_state ORDER BY total_orders DESC;""", conn)

state_orders.to_csv("state_orders_sql.csv", index=False)
files.download("state_orders_sql.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>